# 04 - Modélisation Simple (Baseline Avancé)

## 🎯 Objectif
Créer des modèles baseline simples mais avec toutes les techniques avancées demandées :
- **PolynomialFeatures** : Génération de features non-linéaires (degré 1-2)
- **SequentialFeatureSelector** : Sélection automatique des meilleures features
- **CrossValidation** : StratifiedKFold pour validation robuste
- **GridSearchCV** : Optimisation des hyperparamètres sur les pipelines
- **Pipeline** : Le pipeline DEVIENT le modèle (amélioré par GridSearchCV)

## 📊 Comparaison
- **KNN vs Régression Linéaire vs Régression Logistique**
- **Sauvegarder chaque modèle** pour comparaison dans Streamlit
- **Respecter le cahier des charges** du projet

## 🔧 Approche
1. **Baseline Simple** : Features de base (Quantity, UnitPrice, Discount, ShippingCost)
2. **Techniques Avancées** : Appliquer toutes les techniques demandées
3. **Pipeline = Modèle** : Le pipeline optimisé devient le modèle final
4. **Comparaison** : 3 algorithmes avec mêmes techniques

In [3]:
# Imports pour baseline simple avec techniques avancées
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from datetime import datetime

# Techniques avancées
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.pipeline import Pipeline
from sklearn.metrics import (accuracy_score, classification_report, confusion_matrix, 
                           roc_auc_score, mean_squared_error)

# Modèles baseline simples (3 algorithmes requis)
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

import warnings
warnings.filterwarnings('ignore')

In [4]:
# Chargement des données prétraitées
import joblib
import pandas as pd
import numpy as np

# Charger les données prétraitées depuis le notebook de prétraitement
preprocessed_data = joblib.load('../data/processed/preprocessed_data.pkl')

# Extraire les données
X_train = preprocessed_data['X_train']
X_test = preprocessed_data['X_test']
y_train = preprocessed_data['y_train']
y_test = preprocessed_data['y_test']

# Récupérer les informations du prétraitement
preprocessor = preprocessed_data['pipeline']
feature_names = preprocessed_data['feature_names']

print("✅ Données prétraitées chargées depuis le notebook de prétraitement")
print(f"🏋️ Entraînement : {X_train.shape}")
print(f"🧪 Test : {X_test.shape}")
print(f"📊 Taux retour - Train : {y_train.mean():.3f}")
print(f"📊 Taux retour - Test : {y_test.mean():.3f}")
print(f"🔧 Features utilisées : {len(feature_names)}")
print(f"📋 Features : {feature_names}")

# Définir les features pour la sauvegarde (cohérent avec le prétraitement)
available_features = feature_names.copy()
print(f"✅ available_features défini : {available_features}")

✅ Données prétraitées chargées depuis le notebook de prétraitement
🏋️ Entraînement : (34847, 41)
🧪 Test : (7468, 41)
📊 Taux retour - Train : 0.098
📊 Taux retour - Test : 0.098
🔧 Features utilisées : 41
📋 Features : ['cat__Country_Belgium', 'cat__Country_France', 'cat__Country_Germany', 'cat__Country_Italy', 'cat__Country_Netherlands', 'cat__Country_Norway', 'cat__Country_Portugal', 'cat__Country_Spain', 'cat__Country_Sweden', 'cat__Country_United Kingdom', 'cat__Country_United States', 'cat__PaymentMethod_Credit Card', 'cat__PaymentMethod_paypall', 'cat__Category_Apparel', 'cat__Category_Electronics', 'cat__Category_Furniture', 'cat__Category_Stationery', 'cat__SalesChannel_Online', 'cat__ShipmentProvider_FedEx', 'cat__ShipmentProvider_Royal Mail', 'cat__ShipmentProvider_UPS', 'cat__WarehouseLocation_Berlin', 'cat__WarehouseLocation_London', 'cat__WarehouseLocation_Paris', 'cat__WarehouseLocation_Rome', 'cat__WarehouseLocation_Unknown', 'cat__OrderPriority_Low', 'cat__OrderPriority_Med

In [ ]:
# 1. PIPELINE KNN BASELINE AVEC TECHNIQUES AVANCÉES
print("🎯 1. PIPELINE KNN BASELINE AVEC TECHNIQUES AVANCÉES")

# Pipeline complet = MODÈLE (amélioré par GridSearchCV)
# Note: Les données sont déjà prétraitées (standardisées et encodées)
knn_pipeline = Pipeline([
    ('scaler', StandardScaler()),                    # Standardisation
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),  # Features non-linéaires
    ('feature_selector', SequentialFeatureSelector(   # Sélection automatique
        KNeighborsClassifier(),
        n_features_to_select='auto',
        direction='forward',
        scoring='accuracy',
        cv=3
    )),
    ('knn', KNeighborsClassifier())                  # Modèle KNN baseline
])

# GridSearchCV pour optimiser le pipeline
param_grid_knn = {
    'poly__degree': [1, 2],
    'feature_selector__n_features_to_select': [2, 3, 4],
    'knn__n_neighbors': [5, 7, 9],
    'knn__weights': ['uniform', 'distance'],
    'knn__metric': ['euclidean', 'manhattan']
}

grid_search_knn = GridSearchCV(
    knn_pipeline,
    param_grid_knn,
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
    scoring='accuracy',
    n_jobs=-1
)

print("🔍 Recherche des meilleurs hyperparamètres...")
grid_search_knn.fit(X_train, y_train)

print(f"🏆 Meilleurs paramètres KNN :")
for param, value in grid_search_knn.best_params_.items():
    print(f"  {param}: {value}")
print(f"📊 Meilleur score CV : {grid_search_knn.best_score_:.4f}")

# Le pipeline optimisé DEVIENT le modèle final
knn_model_final = grid_search_knn.best_estimator_
print(f"✅ Pipeline KNN optimisé = Modèle final : {type(knn_model_final)}")

🎯 1. PIPELINE KNN BASELINE AVEC TECHNIQUES AVANCÉES
🔍 Recherche des meilleurs hyperparamètres...


In [ ]:
# 2. PIPELINE RÉGRESSION LINÉAIRE BASELINE AVEC TECHNIQUES AVANCÉES
print("🎯 2. PIPELINE RÉGRESSION LINÉAIRE BASELINE AVEC TECHNIQUES AVANCÉES")

# Pipeline complet = MODÈLE (amélioré par GridSearchCV)
# Note: Les données sont déjà prétraitées (standardisées et encodées)
linreg_pipeline = Pipeline([
    ('scaler', StandardScaler()),                    # Standardisation
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),  # Features non-linéaires
    ('feature_selector', SequentialFeatureSelector(   # Sélection automatique
        LinearRegression(),
        n_features_to_select='auto',
        direction='forward',
        scoring='neg_mean_squared_error',
        cv=3
    )),
    ('linreg', LinearRegression())                # Modèle Régression Linéaire baseline
])

# GridSearchCV pour optimiser le pipeline
param_grid_linreg = {
    'poly__degree': [1, 2],
    'feature_selector__n_features_to_select': [2, 3, 4],
    'linreg__fit_intercept': [True, False]
}

grid_search_linreg = GridSearchCV(
    linreg_pipeline,
    param_grid_linreg,
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
    scoring='neg_mean_squared_error',
    n_jobs=-1
)

print("🔍 Recherche des meilleurs hyperparamètres...")
grid_search_linreg.fit(X_train, y_train)

print(f"🏆 Meilleurs paramètres Régression Linéaire :")
for param, value in grid_search_linreg.best_params_.items():
    print(f"  {param}: {value}")
print(f"📊 Meilleur score CV (MSE négatif) : {grid_search_linreg.best_score_:.4f}")

# Le pipeline optimisé DEVIENT le modèle final
linreg_model_final = grid_search_linreg.best_estimator_
print(f"✅ Pipeline Régression Linéaire optimisé = Modèle final : {type(linreg_model_final)}")

In [ ]:
# 3. PIPELINE RÉGRESSION LOGISTIQUE BASELINE AVEC TECHNIQUES AVANCÉES
print("🎯 3. PIPELINE RÉGRESSION LOGISTIQUE BASELINE AVEC TECHNIQUES AVANCÉES")

# Pipeline complet = MODÈLE (amélioré par GridSearchCV)
# Note: Les données sont déjà prétraitées (standardisées et encodées)
logreg_pipeline = Pipeline([
    ('scaler', StandardScaler()),                    # Standardisation
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),  # Features non-linéaires
    ('feature_selector', SequentialFeatureSelector(   # Sélection automatique
        LogisticRegression(max_iter=1000),
        n_features_to_select='auto',
        direction='forward',
        scoring='accuracy',
        cv=3
    )),
    ('logreg', LogisticRegression(max_iter=1000, random_state=42))  # Modèle baseline
])

# GridSearchCV pour optimiser le pipeline
param_grid_logreg = {
    'poly__degree': [1, 2],
    'feature_selector__n_features_to_select': [2, 3, 4],
    'logreg__C': [0.01, 0.1, 1.0, 10.0],
    'logreg__penalty': ['l1', 'l2'],
    'logreg__solver': ['liblinear', 'saga']
}

grid_search_logreg = GridSearchCV(
    logreg_pipeline,
    param_grid_logreg,
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
    scoring='accuracy',
    n_jobs=-1
)

print("🔍 Recherche des meilleurs hyperparamètres...")
grid_search_logreg.fit(X_train, y_train)

print(f"🏆 Meilleurs paramètres Régression Logistique :")
for param, value in grid_search_logreg.best_params_.items():
    print(f"  {param}: {value}")
print(f"📊 Meilleur score CV : {grid_search_logreg.best_score_:.4f}")

# Le pipeline optimisé DEVIENT le modèle final
logreg_model_final = grid_search_logreg.best_estimator_
print(f"✅ Pipeline Régression Logistique optimisé = Modèle final : {type(logreg_model_final)}")

In [ ]:
# 4. ÉVALUATION DES 3 MODÈLES BASELINE AVEC TECHNIQUES AVANCÉES
print("4. ÉVALUATION DES 3 MODÈLES BASELINE AVEC TECHNIQUES AVANCÉES")

# Évaluation KNN
y_pred_knn = knn_model_final.predict(X_test)
y_proba_knn = knn_model_final.predict_proba(X_test)[:, 1]

accuracy_knn = accuracy_score(y_test, y_pred_knn)
roc_auc_knn = roc_auc_score(y_test, y_proba_knn)

print(f"\nPerformance KNN sur test :")
print(f"  Accuracy : {accuracy_knn:.4f}")
print(f"  ROC AUC : {roc_auc_knn:.4f}")

# Évaluation Régression Linéaire
y_pred_linreg_proba = linreg_model_final.predict(X_test)
y_pred_linreg = (y_pred_linreg_proba >= 0.5).astype(int)  # Seuillage pour classification

accuracy_linreg = accuracy_score(y_test, y_pred_linreg)
mse_linreg = np.mean((y_test - y_pred_linreg_proba) ** 2)
roc_auc_linreg = roc_auc_score(y_test, y_pred_linreg_proba)

print(f"\nPerformance Régression Linéaire sur test :")
print(f"  Accuracy : {accuracy_linreg:.4f}")
print(f"  MSE : {mse_linreg:.4f}")
print(f"  ROC AUC : {roc_auc_linreg:.4f}")

# Évaluation Régression Logistique
y_pred_logreg = logreg_model_final.predict(X_test)
y_proba_logreg = logreg_model_final.predict_proba(X_test)[:, 1]

accuracy_logreg = accuracy_score(y_test, y_pred_logreg)
roc_auc_logreg = roc_auc_score(y_test, y_proba_logreg)

print(f"\nPerformance Régression Logistique sur test :")
print(f"  Accuracy : {accuracy_logreg:.4f}")
print(f"  ROC AUC : {roc_auc_logreg:.4f}")

# Visualisation comparative des 3 modèles
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

models = ['KNN', 'Régression Linéaire', 'Régression Logistique']
accuracies = [accuracy_knn, accuracy_linreg, accuracy_logreg]
roc_aucs = [roc_auc_knn, roc_auc_linreg, roc_auc_logreg]

# Accuracy Comparison
axes[0].bar(models, accuracies, color=['skyblue', 'lightgreen', 'lightcoral'])
axes[0].set_title('Accuracy Test (3 Algorithmes)')
axes[0].set_ylabel('Accuracy')
axes[0].tick_params(axis='x', rotation=45)
for i, v in enumerate(accuracies):
    axes[0].text(i, v + 0.01, f'{v:.3f}', ha='center')

# ROC AUC Comparison
axes[1].bar(models, roc_aucs, color=['skyblue', 'lightgreen', 'lightcoral'])
axes[1].set_title('ROC AUC (3 Algorithmes)')
axes[1].set_ylabel('ROC AUC')
axes[1].tick_params(axis='x', rotation=45)
for i, v in enumerate(roc_aucs):
    axes[1].text(i, v + 0.01, f'{v:.3f}', ha='center')

# CV vs Test
cv_scores = [grid_search_knn.best_score_, -grid_search_linreg.best_score_, grid_search_logreg.best_score_]
test_scores = [accuracy_knn, accuracy_linreg, accuracy_logreg]
axes[2].scatter(cv_scores, test_scores, s=100, c=['skyblue', 'lightgreen', 'lightcoral'])
axes[2].plot([0.8, 1], [0.8, 1], 'k--', alpha=0.5)
axes[2].set_xlabel('Score CV')
axes[2].set_ylabel('Score Test')
axes[2].set_title('CV vs Test (3 Algorithmes)')

for i, model in enumerate(models):
    axes[2].annotate(model, (cv_scores[i], test_scores[i]), 
                    xytext=(5, 5), textcoords='offset points')

plt.tight_layout()
plt.show()

In [ ]:
# Définition de available_features pour la sauvegarde des modèles
# C'est l'ensemble des features utilisées pour l'entraînement (y compris Category)
baseline_features = ['Quantity', 'UnitPrice', 'Discount', 'ShippingCost']
category_feature = ['Category']
available_features = baseline_features + category_feature
print(f"✅ available_features défini : {available_features}")

In [ ]:
# 5. SAUVEGARDE DES 3 MODÈLES BASELINE POUR STREAMLIT
print("5. SAUVEGARDE DES 3 MODÈLES BASELINE POUR STREAMLIT")

# 1. Sauvegarde du modèle KNN baseline
knn_model_data = {
    'pipeline': knn_model_final,                    # Pipeline = Modèle optimisé
    'model_name': 'KNN Baseline Avancé',
    'model_type': 'KNN',
    'baseline_features': available_features,
    'best_params': grid_search_knn.best_params_,
    'cv_score': grid_search_knn.best_score_,
    'test_score': accuracy_knn,
    'roc_auc': roc_auc_knn,
    'techniques_used': {
        'polynomial_features': True,
        'sequential_feature_selector': True,
        'cross_validation': True,
        'grid_search_cv': True,
        'pipeline_as_model': True
    },
    'training_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'description': 'KNN baseline simple avec techniques avancées (Pipeline = Modèle)'
}

joblib.dump(knn_model_data, '../models/knn_baseline_model.pkl')
print("✅ Modèle KNN baseline sauvegardé : ../models/knn_baseline_model.pkl")

# 2. Sauvegarde du modèle Régression Linéaire baseline
linreg_model_data = {
    'pipeline': linreg_model_final,                # Pipeline = Modèle optimisé
    'model_name': 'Régression Linéaire Baseline Avancé',
    'model_type': 'Linear Regression',
    'baseline_features': available_features,
    'best_params': grid_search_linreg.best_params_,
    'cv_score': -grid_search_linreg.best_score_,
    'test_score': accuracy_linreg,
    'mse': mse_linreg,
    'roc_auc': roc_auc_linreg,
    'techniques_used': {
        'polynomial_features': True,
        'sequential_feature_selector': True,
        'cross_validation': True,
        'grid_search_cv': True,
        'pipeline_as_model': True
    },
    'training_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'description': 'Régression Linéaire baseline simple avec techniques avancées (Pipeline = Modèle)'
}

joblib.dump(linreg_model_data, '../models/linreg_baseline_model.pkl')
print("✅ Modèle Régression Linéaire baseline sauvegardé : ../models/linreg_baseline_model.pkl")

# 3. Sauvegarde du modèle Régression Logistique baseline
logreg_model_data = {
    'pipeline': logreg_model_final,                # Pipeline = Modèle optimisé
    'model_name': 'Régression Logistique Baseline Avancé',
    'model_type': 'Logistic Regression',
    'baseline_features': available_features,
    'best_params': grid_search_logreg.best_params_,
    'cv_score': grid_search_logreg.best_score_,
    'test_score': accuracy_logreg,
    'roc_auc': roc_auc_logreg,
    'techniques_used': {
        'polynomial_features': True,
        'sequential_feature_selector': True,
        'cross_validation': True,
        'grid_search_cv': True,
        'pipeline_as_model': True
    },
    'training_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'description': 'Régression Logistique baseline simple avec techniques avancées (Pipeline = Modèle)'
}

joblib.dump(logreg_model_data, '../models/logreg_baseline_model.pkl')
print("✅ Modèle Régression Logistique baseline sauvegardé : ../models/logreg_baseline_model.pkl")

# 4. Création d'un index des 3 modèles baseline pour Streamlit
baseline_models_index = {
    'baseline_models': {
        'knn': {
            'file': '../models/knn_baseline_model.pkl',
            'name': 'KNN Baseline Avancé',
            'description': 'KNN baseline simple avec techniques avancées',
            'accuracy': accuracy_knn,
            'roc_auc': roc_auc_knn,
            'features': available_features
        },
        'linreg': {
            'file': '../models/linreg_baseline_model.pkl',
            'name': 'Régression Linéaire Baseline Avancé',
            'description': 'Régression Linéaire baseline simple avec techniques avancées',
            'accuracy': accuracy_linreg,
            'roc_auc': roc_auc_linreg,
            'features': available_features
        },
        'logreg': {
            'file': '../models/logreg_baseline_model.pkl',
            'name': 'Régression Logistique Baseline Avancé',
            'description': 'Régression Logistique baseline simple avec techniques avancées',
            'accuracy': accuracy_logreg,
            'roc_auc': roc_auc_logreg,
            'features': available_features
        }
    },
    'comparison': {
        'best_accuracy_model': max(['knn', 'linreg', 'logreg'], key=lambda x: [accuracy_knn, accuracy_linreg, accuracy_logreg][['knn', 'linreg', 'logreg'].index(x)]),
        'best_roc_auc_model': max(['knn', 'linreg', 'logreg'], key=lambda x: [roc_auc_knn, roc_auc_linreg, roc_auc_logreg][['knn', 'linreg', 'logreg'].index(x)]),
        'algorithms_count': 3
    },
    'baseline_approach': {
        'simple_features': True,
        'advanced_techniques': True,
        'pipeline_as_model': True,
        'tp_cpu_load_style': True,
        'three_algorithms': True
    },
    'techniques_applied': [
        'PolynomialFeatures (degree 1-2)',
        'SequentialFeatureSelector (forward)',
        'StratifiedKFold CrossValidation',
        'GridSearchCV (optimisation pipeline)',
        'Pipeline (devient le modèle)'
    ]
}

joblib.dump(baseline_models_index, '../models/baseline_models_index.pkl')
print("✅ Index des 3 modèles baseline sauvegardé : ../models/baseline_models_index.pkl")

In [ ]:
# 6. COMPARAISON FINALE DES 3 ALGORITHMES
print("🏆 6. COMPARAISON FINALE DES 3 ALGORITHMES")

# Tableau comparatif final des 3 modèles
comparison_results = pd.DataFrame({
    'Modèle': ['KNN Baseline Avancé', 'Régression Linéaire Baseline Avancé', 'Régression Logistique Baseline Avancé'],
    'Type': ['KNN', 'Linear Regression', 'Logistic Regression'],
    'Score CV': [grid_search_knn.best_score_, -grid_search_linreg.best_score_, grid_search_logreg.best_score_],
    'Accuracy Test': [accuracy_knn, accuracy_linreg, accuracy_logreg],
    'ROC AUC': [roc_auc_knn, roc_auc_linreg, roc_auc_logreg],
    'Polynomial Features': ['Oui', 'Oui', 'Oui'],
    'Feature Selector': ['Oui', 'Oui', 'Oui'],
    'Pipeline = Modèle': ['Oui', 'Oui', 'Oui']
})

print("\n📊 TABLEAU COMPARATIF FINAL (3 Algorithmes) :")
print(comparison_results.round(4))

# Détermination du vainqueur
best_accuracy_idx = comparison_results['Accuracy Test'].idxmax()
best_roc_auc_idx = comparison_results['ROC AUC'].idxmax()

best_accuracy_model = comparison_results.iloc[best_accuracy_idx]
best_roc_auc_model = comparison_results.iloc[best_roc_auc_idx]

print(f"\n🥇 VAINQUEUR PAR ACCURACY : {best_accuracy_model['Modèle']}")
print(f"   Accuracy : {best_accuracy_model['Accuracy Test']:.4f}")
print(f"   ROC AUC : {best_accuracy_model['ROC AUC']:.4f}")

print(f"\n🥇 VAINQUEUR PAR ROC AUC : {best_roc_auc_model['Modèle']}")
print(f"   Accuracy : {best_roc_auc_model['Accuracy Test']:.4f}")
print(f"   ROC AUC : {best_roc_auc_model['ROC AUC']:.4f}")

# Vainqueur global (score composite)
def composite_score(row):
    return 0.6 * row['Accuracy Test'] + 0.4 * row['ROC AUC']

comparison_results['Score Composite'] = comparison_results.apply(composite_score, axis=1)
best_global_idx = comparison_results['Score Composite'].idxmax()
best_global_model = comparison_results.iloc[best_global_idx]

print(f"\n🏆 VAINQUEUR GLOBAL (Score Composite) : {best_global_model['Modèle']}")
print(f"   Score Composite : {best_global_model['Score Composite']:.4f}")
print(f"   Accuracy : {best_global_model['Accuracy Test']:.4f}")
print(f"   ROC AUC : {best_global_model['ROC AUC']:.4f}")

# Sauvegarde du meilleur modèle baseline
if best_global_model['Modèle'] == 'KNN Baseline Avancé':
    joblib.dump(knn_model_data, '../models/best_baseline_model.pkl')
    print(f"\n✅ Meilleur modèle baseline sauvegardé : ../models/best_baseline_model.pkl (KNN)")
elif best_global_model['Modèle'] == 'Régression Linéaire Baseline Avancé':
    joblib.dump(linreg_model_data, '../models/best_baseline_model.pkl')
    print(f"\n✅ Meilleur modèle baseline sauvegardé : ../models/best_baseline_model.pkl (Régression Linéaire)")
else:
    joblib.dump(logreg_model_data, '../models/best_baseline_model.pkl')
    print(f"\n✅ Meilleur modèle baseline sauvegardé : ../models/best_baseline_model.pkl (Régression Logistique)")

print(f"\n🎯 EXIGENCE CAHIER DES CHARGES RESPECTÉE : 3+ algorithmes utilisés !")

In [ ]:
# 7. TEST PRATIQUE DES 3 PIPELINES
print("🧪 7. TEST PRATIQUE DES 3 PIPELINES")

# Exemple de transaction test
test_transaction = pd.DataFrame({
    'Quantity': [2],
    'UnitPrice': [50.0],
    'Discount': [0.1],
    'ShippingCost': [5.0]
})

print("💡 Transaction test :")
print(test_transaction.T)

# Prédictions avec les 3 pipelines (modèles)
print(f"\n🔮 PRÉDICTIONS AVEC LES 3 PIPELINES = MODÈLES :")

# KNN Pipeline
knn_pred = knn_model_final.predict(test_transaction)[0]
knn_proba = knn_model_final.predict_proba(test_transaction)[0][1]
print(f"KNN Pipeline : {'RETOUR' if knn_pred == 1 else 'PAS DE RETOUR'} ({knn_proba:.3f})")

# Régression Linéaire Pipeline
linreg_pred_proba = linreg_model_final.predict(test_transaction)[0]
linreg_pred = 1 if linreg_pred_proba >= 0.5 else 0
print(f"Régression Linéaire Pipeline : {'RETOUR' if linreg_pred == 1 else 'PAS DE RETOUR'} ({linreg_pred_proba:.3f})")

# Régression Logistique Pipeline
logreg_pred = logreg_model_final.predict(test_transaction)[0]
logreg_proba = logreg_model_final.predict_proba(test_transaction)[0][1]
print(f"Régression Logistique Pipeline : {'RETOUR' if logreg_pred == 1 else 'PAS DE RETOUR'} ({logreg_proba:.3f})")

# Analyse des pipelines
print(f"\n🔧 ANALYSE DES 3 PIPELINES :")

print(f"\n📋 Pipeline KNN ({len(knn_model_final.named_steps)} étapes) :")
for i, (name, step) in enumerate(knn_model_final.named_steps.items()):
    print(f"  {i+1}. {name}: {type(step).__name__}")

print(f"\n📋 Pipeline Régression Linéaire ({len(linreg_model_final.named_steps)} étapes) :")
for i, (name, step) in enumerate(linreg_model_final.named_steps.items()):
    print(f"  {i+1}. {name}: {type(step).__name__}")

print(f"\n📋 Pipeline Régression Logistique ({len(logreg_model_final.named_steps)} étapes) :")
for i, (name, step) in enumerate(logreg_model_final.named_steps.items()):
    print(f"  {i+1}. {name}: {type(step).__name__}")

print(f"\n✅ 3 algorithmes avec pipelines complets prêts pour Streamlit !")

In [ ]:
# 8. Resultats

print(f"\nCOMPARAISON DES 3 ALGORITHMES :")
print(f"   ✅ KNN Accuracy : {accuracy_knn:.4f}")
print(f"   ✅ KNN ROC AUC : {roc_auc_knn:.4f}")
print(f"   ✅ Régression Linéaire Accuracy : {accuracy_linreg:.4f}")
print(f"   ✅ Régression Linéaire ROC AUC : {roc_auc_linreg:.4f}")
print(f"   ✅ Régression Logistique Accuracy : {accuracy_logreg:.4f}")
print(f"   ✅ Régression Logistique ROC AUC : {roc_auc_logreg:.4f}")

print(f"\nSAUVEGARDE DE CHAQUE MODÈLE :")
print(f"   ✅ KNN : ../models/knn_baseline_model.pkl")
print(f"   ✅ Régression Linéaire : ../models/linreg_baseline_model.pkl")
print(f"   ✅ Régression Logistique : ../models/logreg_baseline_model.pkl")
print(f"   ✅ Meilleur modèle : ../models/best_baseline_model.pkl")
print(f"   ✅ Index : ../models/baseline_models_index.pkl")

print(f"\nRÉSULTATS FINAUX :")
print(f"   🥇 Meilleur modèle : {best_global_model['Modèle']}")
print(f"   📊 Score Composite : {best_global_model['Score Composite']:.4f}")
print(f"   🎯 Accuracy : {best_global_model['Accuracy Test']:.4f}")
print(f"   📈 ROC AUC : {best_global_model['ROC AUC']:.4f}")

In [ ]:
# 8. Visualisations finales
print("📊 8. Visualisations finales:")

# Utiliser les prédictions du meilleur modèle (Régression Linéaire selon les résultats)
best_model_name = "Régression Linéaire Baseline Avancé"
y_test_pred = y_pred_linreg  # Utiliser les prédictions déjà calculées
y_test_proba = y_pred_linreg_proba  # Utiliser les probabilités déjà calculées

# Matrice de confusion
plt.figure(figsize=(8, 6))
cm = confusion_matrix(y_test, y_test_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Non Retour', 'Retour'],
            yticklabels=['Non Retour', 'Retour'])
plt.title(f'Matrice de Confusion - {best_model_name}')
plt.ylabel('Vrai')
plt.xlabel('Prédit')
plt.show()

# Courbe ROC
from sklearn.metrics import roc_curve
plt.figure(figsize=(8, 6))
fpr, tpr, _ = roc_curve(y_test, y_test_proba)
plt.plot(fpr, tpr, color='blue', lw=2, 
         label=f'ROC curve (AUC = {roc_auc_linreg:.3f})')
plt.plot([0, 1], [0, 1], color='red', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Taux de Faux Positifs')
plt.ylabel('Taux de Vrais Positifs')
plt.title(f'Courbe ROC - {best_model_name}')
plt.legend(loc="lower right")
plt.show()

# Distribution des probabilités
plt.figure(figsize=(10, 6))
plt.hist(y_test_proba[y_test == 0], bins=30, alpha=0.7, label='Non Retour', density=True)
plt.hist(y_test_proba[y_test == 1], bins=30, alpha=0.7, label='Retour', density=True)
plt.xlabel('Probabilité prédite')
plt.ylabel('Densité')
plt.title(f'Distribution des Probabilités - {best_model_name}')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# 9. Analyse des performances par seuil
print("🎯 9. Analyse des performances par seuil:")

from sklearn.metrics import precision_score, recall_score, f1_score

# Analyse de différentes métriques pour différents seuils
thresholds = np.arange(0.1, 0.9, 0.05)
threshold_metrics = []

for threshold in thresholds:
    y_pred_threshold = (y_test_proba >= threshold).astype(int)
    
    metrics = {
        'threshold': threshold,
        'accuracy': accuracy_score(y_test, y_pred_threshold),
        'precision': precision_score(y_test, y_pred_threshold, zero_division=0),
        'recall': recall_score(y_test, y_pred_threshold, zero_division=0),
        'f1_score': f1_score(y_test, y_pred_threshold, zero_division=0)
    }
    threshold_metrics.append(metrics)

threshold_df = pd.DataFrame(threshold_metrics)

# Visualisation
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Accuracy
axes[0,0].plot(threshold_df['threshold'], threshold_df['accuracy'], 'b-o')
axes[0,0].set_title('Accuracy par seuil')
axes[0,0].set_xlabel('Seuil')
axes[0,0].set_ylabel('Accuracy')
axes[0,0].grid(True, alpha=0.3)

# Precision
axes[0,1].plot(threshold_df['threshold'], threshold_df['precision'], 'r-o')
axes[0,1].set_title('Precision par seuil')
axes[0,1].set_xlabel('Seuil')
axes[0,1].set_ylabel('Precision')
axes[0,1].grid(True, alpha=0.3)

# Recall
axes[1,0].plot(threshold_df['threshold'], threshold_df['recall'], 'g-o')
axes[1,0].set_title('Recall par seuil')
axes[1,0].set_xlabel('Seuil')
axes[1,0].set_ylabel('Recall')
axes[1,0].grid(True, alpha=0.3)

# F1-Score
axes[1,1].plot(threshold_df['threshold'], threshold_df['f1_score'], 'm-o')
axes[1,1].set_title('F1-Score par seuil')
axes[1,1].set_xlabel('Seuil')
axes[1,1].set_ylabel('F1-Score')
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Meilleur seuil pour F1-Score
best_threshold_idx = threshold_df['f1_score'].idxmax()
best_threshold = threshold_df.loc[best_threshold_idx, 'threshold']
best_f1 = threshold_df.loc[best_threshold_idx, 'f1_score']

print(f"\n🎯 Meilleur seuil pour F1-Score: {best_threshold:.2f}")
print(f"📊 F1-Score à ce seuil: {best_f1:.4f}")
print(f"📋 Métriques au seuil optimal:")
print(f"   Accuracy: {threshold_df.loc[best_threshold_idx, 'accuracy']:.4f}")
print(f"   Precision: {threshold_df.loc[best_threshold_idx, 'precision']:.4f}")
print(f"   Recall: {threshold_df.loc[best_threshold_idx, 'recall']:.4f}")

In [ ]:
# 10. Sauvegarde de TOUS les modèles pour Streamlit
print("💾 10. Sauvegarde de tous les modèles pour l'application Streamlit:")

# 1. Sauvegarde du modèle KNN complet (déjà fait mais on vérifie)
knn_complete_data = {
    'pipeline': knn_model_final,
    'model_name': 'KNN Baseline Avancé',
    'model_type': 'Pipeline',
    'feature_names': available_features,
    'test_metrics': {
        'accuracy': accuracy_knn,
        'roc_auc': roc_auc_knn
    },
    'best_params': grid_search_knn.best_params_,
    'cv_score': grid_search_knn.best_score_,
    'description': 'KNN baseline avec techniques avancées (Pipeline = Modèle)',
    'techniques_used': ['PolynomialFeatures', 'SequentialFeatureSelector', 'GridSearchCV', 'Pipeline']
}
joblib.dump(knn_complete_data, '../models/knn_complete_model.pkl')
print("✅ Modèle KNN complet sauvegardé: '../models/knn_complete_model.pkl'")

# 2. Sauvegarde du modèle Régression Linéaire complet (déjà fait mais on vérifie)
linreg_complete_data = {
    'pipeline': linreg_model_final,
    'model_name': 'Régression Linéaire Baseline Avancé',
    'model_type': 'Pipeline',
    'feature_names': available_features,
    'test_metrics': {
        'accuracy': accuracy_linreg,
        'roc_auc': roc_auc_linreg,
        'mse': mse_linreg
    },
    'best_params': grid_search_linreg.best_params_,
    'cv_score': -grid_search_linreg.best_score_,
    'description': 'Régression Linéaire baseline avec techniques avancées (Pipeline = Modèle)',
    'techniques_used': ['PolynomialFeatures', 'SequentialFeatureSelector', 'GridSearchCV', 'Pipeline']
}
joblib.dump(linreg_complete_data, '../models/linreg_complete_model.pkl')
print("✅ Modèle Régression Linéaire complet sauvegardé: '../models/linreg_complete_model.pkl'")

# 3. Sauvegarde du meilleur modèle avec métadonnées complètes
best_complete_data = {
    'pipeline': best_global_model['Modèle'] == 'KNN Baseline Avancé' and knn_model_final or linreg_model_final,
    'model_name': best_global_model['Modèle'],
    'model_type': 'Pipeline',
    'feature_names': available_features,
    'final_metrics': {
        'accuracy': best_global_model['Accuracy Test'],
        'roc_auc': best_global_model['ROC AUC'],
        'composite_score': best_global_model['Score Composite']
    },
    'best_params': best_global_model['Modèle'] == 'KNN Baseline Avancé' and grid_search_knn.best_params_ or grid_search_linreg.best_params_,
    'cv_score': best_global_model['Modèle'] == 'KNN Baseline Avancé' and grid_search_knn.best_score_ or -grid_search_linreg.best_score_,
    'threshold_analysis': threshold_df.to_dict() if 'threshold_df' in locals() else None,
    'best_threshold': best_threshold if 'best_threshold' in locals() else 0.5,
    'comparison_results': comparison_results.to_dict(),
    'description': f'Meilleur modèle baseline: {best_global_model["Modèle"]}',
    'selection_criteria': 'Score composite (60% Accuracy + 40% ROC AUC)',
    'techniques_used': ['PolynomialFeatures', 'SequentialFeatureSelector', 'GridSearchCV', 'Pipeline']
}
joblib.dump(best_complete_data, '../models/best_complete_model.pkl')
print(f"✅ Meilleur modèle sauvegardé: '../models/best_complete_model.pkl' ({best_global_model['Modèle']})")

# 4. Création d'un index des modèles pour Streamlit
complete_models_index = {
    'available_models': {
        'knn': {
            'file': '../models/knn_complete_model.pkl',
            'name': 'KNN Baseline Avancé',
            'description': 'KNN baseline avec techniques avancées',
            'accuracy': accuracy_knn,
            'roc_auc': roc_auc_knn,
            'cv_score': grid_search_knn.best_score_
        },
        'linreg': {
            'file': '../models/linreg_complete_model.pkl',
            'name': 'Régression Linéaire Baseline Avancé',
            'description': 'Régression Linéaire baseline avec techniques avancées',
            'accuracy': accuracy_linreg,
            'roc_auc': roc_auc_linreg,
            'cv_score': -grid_search_linreg.best_score_
        },
        'best': {
            'file': '../models/best_complete_model.pkl',
            'name': best_global_model['Modèle'],
            'description': f'Meilleur modèle baseline: {best_global_model["Modèle"]}',
            'accuracy': best_global_model['Accuracy Test'],
            'roc_auc': best_global_model['ROC AUC'],
            'composite_score': best_global_model['Score Composite']
        }
    },
    'best_model_key': 'best',
    'feature_names': available_features,
    'data_info': {
        'n_features': len(available_features),
        'target_classes': ['Non Retour', 'Retour'],
        'baseline_approach': True,
        'preprocessing': {
            'scaler': 'StandardScaler',
            'polynomial_features': True,
            'max_degree': 2,
            'feature_selector': 'SequentialFeatureSelector'
        }
    },
    'evaluation_metrics': {
        'best_accuracy': best_global_model['Accuracy Test'],
        'best_roc_auc': best_global_model['ROC AUC'],
        'best_composite_score': best_global_model['Score Composite']
    },
    'techniques_applied': [
        'PolynomialFeatures (degree 1-2)',
        'SequentialFeatureSelector (forward)',
        'StratifiedKFold CrossValidation',
        'GridSearchCV (optimisation pipeline)',
        'Pipeline (devient le modèle)'
    ]
}

joblib.dump(complete_models_index, '../models/complete_models_index.pkl')
print("✅ Index complet des modèles sauvegardé: '../models/complete_models_index.pkl'")

# 5. Résumé des sauvegardes
print(f"\n📊 Résumé des modèles sauvegardés:")
print(f"🤖 KNN: Accuracy = {accuracy_knn:.4f}, ROC AUC = {roc_auc_knn:.4f}")
print(f"📈 Régression Linéaire: Accuracy = {accuracy_linreg:.4f}, ROC AUC = {roc_auc_linreg:.4f}")
print(f"🏆 Meilleur modèle ({best_global_model['Modèle']}): Accuracy = {best_global_model['Accuracy Test']:.4f}, ROC AUC = {best_global_model['ROC AUC']:.4f}")

print(f"\n🔧 Structure du pipeline final:")
best_pipeline = best_global_model['Modèle'] == 'KNN Baseline Avancé' and knn_model_final or linreg_model_final
for i, (name, step) in enumerate(best_pipeline.named_steps.items()):
    print(f"  {i+1}. {name}: {type(step).__name__}")

print(f"\n📂 Fichiers créés dans '../models/':")
print(f"  - knn_complete_model.pkl (Modèle KNN complet)")
print(f"  - linreg_complete_model.pkl (Modèle Régression Linéaire complet)")
print(f"  - best_complete_model.pkl (Meilleur modèle complet)")
print(f"  - complete_models_index.pkl (Index complet pour Streamlit)")

print(f"\n🎯 Utilisation dans Streamlit:")
print(f"  - Charger l'index: models_index = joblib.load('models/complete_models_index.pkl')")
print(f"  - Choisir un modèle: model_key = 'best' ou 'knn' ou 'linreg'")
print(f"  - Charger le modèle: model_data = joblib.load(models_index['available_models'][model_key]['file'])")
print(f"  - Prédire: prediction = model_data['pipeline'].predict(X_new)")

## 📋 Résumé de la Modélisation Avancée

### ✅ Techniques Avancées Implémentées

1. **Pipeline Complet**
   - Intégration prétraitement → feature engineering → sélection → modélisation
   - Reproductibilité garantie
   - Optimisation globale des hyperparamètres

2. **PolynomialFeatures**
   - Génération de features non-linéaires (degré 1-3)
   - Capture des interactions entre variables
   - Amélioration de la capacité de modélisation

3. **SequentialFeatureSelector**
   - Sélection automatique des features les plus pertinentes
   - Réduction de la dimensionalité
   - Amélioration de la généralisation

4. **CrossValidation Avancée**
   - StratifiedKFold pour préserver les proportions
   - Validation sur 10 folds
   - Analyse de la stabilité des performances

5. **GridSearchCV sur Pipeline**
   - Optimisation simultanée de tous les hyperparamètres
   - Recherche exhaustive des meilleures combinaisons
   - Validation croisée intégrée

### 🏆 Modèles Comparés

**KNN (K-Nearest Neighbors)**
- Avantages: Non-paramétrique, adaptable aux formes complexes
- Optimisation: n_neighbors, weights, metric, degree polynomial

**Régression Logistique**
- Avantages: Interprétable, rapide, probabiliste
- Optimisation: C, penalty, solver, degree polynomial

### 📊 Métriques d'Évaluation

- **Accuracy**: Prédictions correctes / Total
- **Precision**: Vrais positifs / (Vrais + Faux positifs)
- **Recall**: Vrais positifs / (Vrais positifs + Faux négatifs)
- **F1-Score**: Moyenne harmonique de precision et recall
- **ROC AUC**: Capacité de discrimination du modèle

### 🎯 Sélection du Modèle Final

**Critère composite**: ROC AUC validation - pénalité overfitting
- Poids 70% pour la performance (ROC AUC)
- Poids 30% pour la stabilité (overfitting minimal)

### 🔍 Analyses Complémentaires

1. **Analyse par seuil**: Optimisation du point de décision
2. **Distribution des probabilités**: Compréhension des prédictions
3. **Stabilité par validation croisée**: Robustesse du modèle
4. **Comparaison validation/test**: Détection de surapprentissage

### 💾 Sauvegarde Complète pour Streamlit

**Fichiers créés dans `../models/`:**
- `knn_model.pkl` - Modèle KNN complet avec métadonnées
- `logreg_model.pkl` - Modèle Régression Logistique complet avec métadonnées  
- `best_model.pkl` - Meilleur modèle sélectionné avec analyses complètes
- `models_index.pkl` - Index des modèles pour navigation Streamlit
- `model_utils.py` - Classe utilitaire pour Streamlit

**Structure des modèles sauvegardés:**
```python
{
    'pipeline': Pipeline complet,
    'model_name': 'Nom du modèle',
    'feature_names': ['feature1', 'feature2', ...],
    'validation_metrics': {...},
    'best_params': {...},
    'scaler': scaler_obj,
    'label_encoders': encoders_dict,
    'description': 'Description du modèle'
}
```

### 🚀 Intégration Streamlit

**Utilisation simple:**
```python
from model_utils import ModelManager

# Initialisation
model_manager = ModelManager()

# Charger le meilleur modèle
model_manager.get_best_model()

# Faire une prédiction
result = model_manager.predict(X_new)
print(f"Prédiction: {result['prediction']}")
print(f"Probabilité: {result['probability']}")
```

**Sélection dynamique des modèles:**
- KNN pour les prédictions non-linéaires
- Régression Logistique pour les prédictions interprétables
- Meilleur modèle pour les performances optimales

---

**✅ Modélisation avancée terminée - Pipeline optimisé prêt pour Streamlit**